# Predicting ECG Beats 
**Final Project Optimization and Learning**

*Introduction*  \\

Electrocardiograms are obtained on almost all patients who enter the hospital. It provides information about the electrical activity in the heart and is a crucial tool in medical care. Clinicians are taught to recognize and diagnose arrhythmias quickly from ECGs. However, clinicians are often busy with other tasks and may not always be present to monitor a patient's ECG. Automated detection of anomalies is an active area of research to better alert support staff and also for prognostic purposes. For instance, ventricular premature beats (VPB) are typically benign and can occur in health individuals. However, studies have demonstrated associations between VPBs and increased mortality, and VPBs may trigger other more dangerous arrhythmias. Being able to differentiate between VPBs and normal beats automatically may provide some clinical utility. 


# Background
For my final project, I'll be constructing a bilevel optimization problem using a logistic regression model with Ridge regression. I'll be working on the MIT Arrhythmia database where I've acquired 48 ECG traces from 47 patients. For each ECG trace, each beat has been annotated by an expert. To generate have a rich enough dataset, I decide to take each beat as a data sample and extract features from each sample to standardize the size of my inputs. I used the open source available hctsa package to extract 800 features.

In [0]:
# Final Project Optimization and Learning
# Creates a logistic regression model using Ridge regularization
# 
!pip3 install pyomo # Install pyomo locally without printing out statement
!apt-get install -y -qq glpk-utils
!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip" && unzip -o -q couenne-linux64
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip" && unzip -o -q ipopt-linux64


# Import of the pyomo module
from pyomo.opt import SolverFactory
# Import useful modules
import os, sys, math, logging, shutil, time
# Import numpy
from __future__ import division
import numpy as np
import pyomo.environ as pyo
import scipy

    100% |████████████████████████████████| 2.1MB 15.2MB/s 
    100% |████████████████████████████████| 256kB 37.9MB/s 
    100% |████████████████████████████████| 51kB 23.6MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_

In [0]:
# Importing Data from local
# The importance of this was to load in PVCdata.mat
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('/content/gdrive')
print(os.getcwd())


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive


In [0]:
# Here we load in our data: PVCdata.mat

from scipy.io import loadmat
from sklearn.model_selection import train_test_split

DATA = loadmat('/content/gdrive/My Drive/OL_FinalProject/PVCdata.mat')
X = DATA['X'] # Each row represents a different beat
Y = DATA['Y']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.98, random_state=1)

X_sets = np.random.permutation(np.arange(0,len(X_train)))

# We'll perform cross-validation using 10 sets, 9 to train, 1 to validate each round
m = len(Y_train)

print(np.shape(X_train))
print(np.shape(Y_train))

(318, 800)
(318, 1)


# Logistic Regression with Ridge Regression
As we previously discussed in class, logistic regression solves a binary classification problem where:
$$P(x_i) = \dfrac{1}{1+exp(-w^Tx_i)} $$ 
refers to the probability that beat $i$ is a normal beat. We would like to devise a binary classification problem where given a dataset consisting of normal ECG beats and abnormal premature ventricular beats, can we figure out which beats are normal. 
 





---



---



In [0]:

# Initialize model:
model = pyo.ConcreteModel()

# Create Variables:
num_pts = len(Y_train)-1
y_tmp=(Y_train[0:num_pts])
y = {}
m = len(Y_train[0:num_pts])
for i in range(m):
  y[i] = np.asscalar(Y_train[i])

d = 50  
x = {}
for i in range(m):
  for j in range(d):
    x[i,j] = np.asscalar(X_train[i,j])

#x=X_train[0:200,0:10]
model.I = pyo.Set(initialize=range(m)) # Number of patients
model.J = pyo.Set(initialize=range(d)) # Number of features
model.X = pyo.Param(model.I,model.J,initialize=x)
model.Y = pyo.Param(model.I,initialize=y)
#model.num_feat = Param(range(len(model.X)) # Number of features
#print(np.shape(model.I))
model.w  = pyo.Var(model.J)
model.L  = pyo.Var(domain = pyo.NonNegativeReals)

# Set up our constraints for our logistic regression
model.constraints = pyo.ConstraintList()
def constraint_rule1(model,j): # Lower Level Constraints 
  m = len(model.Y)
  print('Constraint ' + str(j) + ' of ' + str(len(model.w)))
  expr = {}
  #for k in model.I:
  #  expr[n] = pyo.sum(pyo.prod(model.X[n,k] for n in model.X,model.w[k]))
  for n in range(len(model.Y)):
    expr_k = 0
    for k in range(len(model.w)):
      expr_k += model.X[n,k]*model.w[k]
    expr[n] = expr_k
  #print('Debugger')
  #print(expr)
  summation=-(1/m)*sum(model.Y[n]/(1+pyo.exp(-model.Y[n]*expr[n]))*model.X[n,j] for n in (model.I))
  #print("Summation Term")
  #print(summation)
  #print(model.L*model.w[j])
  #print(summation + 2*model.L*model.w[j] == 0)
  return summation + 2*model.L*model.w[j] == 0
#print(np.shape(x))

print('Beginning constraints assignment...')
model.LLConstraint = pyo.Constraint(model.J,rule=constraint_rule1)
print('Completed constraint assignment.')


Beginning constraints assignment...
Constraint 0 of 50
Constraint 1 of 50
Constraint 2 of 50
Constraint 3 of 50
Constraint 4 of 50
Constraint 5 of 50
Constraint 6 of 50
Constraint 7 of 50
Constraint 8 of 50
Constraint 9 of 50
Constraint 10 of 50
Constraint 11 of 50
Constraint 12 of 50
Constraint 13 of 50
Constraint 14 of 50
Constraint 15 of 50
Constraint 16 of 50
Constraint 17 of 50
Constraint 18 of 50
Constraint 19 of 50
Constraint 20 of 50
Constraint 21 of 50
Constraint 22 of 50
Constraint 23 of 50
Constraint 24 of 50
Constraint 25 of 50
Constraint 26 of 50
Constraint 27 of 50
Constraint 28 of 50
Constraint 29 of 50
Constraint 30 of 50
Constraint 31 of 50
Constraint 32 of 50
Constraint 33 of 50
Constraint 34 of 50
Constraint 35 of 50
Constraint 36 of 50
Constraint 37 of 50
Constraint 38 of 50
Constraint 39 of 50
Constraint 40 of 50
Constraint 41 of 50
Constraint 42 of 50
Constraint 43 of 50
Constraint 44 of 50
Constraint 45 of 50
Constraint 46 of 50
Constraint 47 of 50
Constraint 48 

# Loss Function
The F1-score metric is normally non-differentiable. However, we can relax this to make the F1 score differentiable. From what we talked about in class with the Brier score and looking at this post: (https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric), I come up with the modified F1 score. Instead of having a decision rule to figure out whether our prediction is 1 or 0, we can instead formulate two cases. First, if a given true label $y_i$ is 1 (a normal beat) and our prediction probability is 0.25, then we would say that the false positive is 0.25 and the true negative is 0.75. In the case where the true label $y_i$ is 0 and our prediction probability is 0.45, then we would say our true negative is 0.55 and our false positive is 0.45. 

In [0]:
# We will now define the functions necesary for the F1 Score to be calculated
# We will use the modified F1 score metric to be better compatible with

def precision(x,y,w):
  Px = np.exp(x.dot(w))>0.5
  V = (y==1)*(Px==1) # This corresponds to the true positives
  TP = V.sum()
  U = (y==0)*(Px==1) # This corresponds to false positives
  FP = U.sum()
  prec = TP/(TP+FP)
  return prec
def recall(x,y,w):
  Px = np.exp(x.dot(w))>0.5
  V = (y==1)*(Px==1) # True positives
  U = (y==1)*(Px==0) # False negatives
  TP = V.sum()
  FN = U.sum()
  prec = TP/(TP+FN)
  return prec
def F1_score(model): 
  # Make this compatible with pyomo
  # Instead of x,y,w we now have model.X,model.Y, and model.w
  expr_TP = {} # Here we'll create an expression for prediction
  expr_FN = {}
  expr_FP = {}
  expr_TN = {}
  # n indexes the beat number, k indexes the features
  m = len(model.Y) # Number of beats considered
  for n in range(m):
    expr_k = 0
    for k in range(len(model.w)):
      expr_k += model.X[n,k]*model.w[k]
    expr_TP[n] = model.Y[n]*(1/(1+pyo.exp(-expr_k)))
    expr_FN[n] = model.Y[n]*(pyo.exp(-expr_k)/(1+pyo.exp(-expr_k)))
    expr_TN[n] =  (1-model.Y[n])*(pyo.exp(-expr_k)/(1+pyo.exp(-expr_k)))
    expr_FP[n] = (1-model.Y[n])*(1/(1+pyo.exp(-expr_k)))
  
  # Perform the calculation now  
  
  precision = sum(expr_TP)/(sum(expr_TP) + sum(expr_FP))
  recall = sum(expr_TP)/(sum(expr_TP) + sum(expr_FN) )

  F1 = 2*precision*recall/(precision+recall)
  return F1
def F1_crossVal(x,y,w):
  # Build this outside around the model.objective function instead
  T = 10
  lenx = len(x)
  F1_t = np.zeros((T,1))
  bins = np.arange(0,lenx,round(lenx/(T+1)))
  bins[-1] = lenx # Adjust the last element
  #print(np.shape(bins))
  c = 0 # Counter index
  for i in np.arange(0,T): # Treat set i as the validation set
    xi = x[bins[i]:bins[i+1],:]
    yi = y[bins[i]:bins[i+1]]
    F1_t[c] = F1_score(xi,yi,w)
    print(F1_t[c])
    c += 1
  F1_avg = (1/T)*F1_t.sum()
  return F1_avg
def sig(x,y,w):
  # Assume y is a scalar and x is a vector
  out = (1+math.e**(-y*(x.dot(w))))**(-1)
  return out
def sig_sum(x,y,w):
  # x is a matrix, y is a vector, w is a vector               
  out_vec = np.zeros((1,len(x[0,:])))
  for i in np.arange(0,len(x)):
    out_vec += -y[i]*sig(x[[i],:],y[i],w)*x[i,:]
  out = 1/len(x)*out_vec
  return out
def lowerLevel_cons(x,y,w,L):
  # Lower level constraints function
  # For scipy, x is a matrix, y is a vector, w is a vector
  out_vector = sig_sum(x,y,w) + w*L
  return out_vector
def upperLevel_obj(x,y,w):
  m = len(x)
  val = (1/m)*(1-F1_score(x,y,w))
  return val
model.objective = pyo.Objective(expr=(1/m)*(1-F1_score(model)),sense=pyo.minimize)

{0: 0, 1: <pyomo.core.expr.expr_pyomo5.ReciprocalExpression object at 0x7f52cfd435e8>, 2: 0, 3: 0, 4: 0, 5: <pyomo.core.expr.expr_pyomo5.ReciprocalExpression object at 0x7f52cfd377f8>, 6: <pyomo.core.expr.expr_pyomo5.ReciprocalExpression object at 0x7f52cfd329a8>, 7: 0, 8: 0, 9: <pyomo.core.expr.expr_pyomo5.ReciprocalExpression object at 0x7f52cfd28e88>, 10: 0, 11: <pyomo.core.expr.expr_pyomo5.ReciprocalExpression object at 0x7f52cfd1de88>, 12: 0, 13: <pyomo.core.expr.expr_pyomo5.ReciprocalExpression object at 0x7f52cfd181f8>, 14: <pyomo.core.expr.expr_pyomo5.ReciprocalExpression object at 0x7f52cfd105b8>, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: <pyomo.core.expr.expr_pyomo5.ReciprocalExpression object at 0x7f52cfd77e58>, 22: <pyomo.core.expr.expr_pyomo5.ReciprocalExpression object at 0x7f52cfd73678>, 23: 0, 24: 0, 25: 0, 26: <pyomo.core.expr.expr_pyomo5.ReciprocalExpression object at 0x7f52cfd64468>, 27: 0, 28: <pyomo.core.expr.expr_pyomo5.ReciprocalExpression object at 0x7f52cfd

In [0]:
# Send the job to Neos to solve
#opt = SolverFactory("conopt")
#solver_manager = pyo.SolverManagerFactory('neos')
#results = solver_manager.solve(model, opt=opt)

# Call the Solver and Display the Results:
solver_choice = pyo.SolverFactory("couenne", executable = "/content/couenne")

results = solver_choice.solve(model)
if str(results.Solver.status) == "ok":
  print("Results\n-------------")
  model.pprint()
model.solutions.store_to(results)
print("\n ")
print("Summarized Results\n-------------")
print(results)



    model=unknown;
        message from solver=couenne\x3a Infeasible problem

 
Summarized Results
-------------

Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 0
  Sense: unknown
Solver: 
- Status: warning
  Message: couenne\x3a Infeasible problem
  Termination condition: infeasible
  Id: 220
  Error rc: 0
  Time: 37.980172634124756
Solution: 
- number of solutions: 1
  number of solutions displayed: 1
- Gap: None
  Status: infeasible
  Message: couenne\x3a Infeasible problem
  Objective:
    objective:
      Value: 0.0015772870662460567
  Variable: No values
  Constraint: No values

